# 11: Shrnutí učiva
---

Obsah:
- Shrnutí učiva
- Materiály k zamyšlení a samostatnému rozvoji znalostí

## Témata ke dalšímu studiu

Doporučuji podívat se na na Minimal Web Api pro tvorbu webových služeb, tedy základu současného internetu:

[Tutorial: Create a minimal API with ASP.NET Core](https://learn.microsoft.com/en-us/aspnet/core/tutorials/min-web-api?view=aspnetcore-7.0&tabs=visual-studio)

Potom se můžete seznámit s tím, jak provádět HTTP requesty s pomocí třídy HttpClient:

[Make HTTP requests with the HttpClient class](https://learn.microsoft.com/en-us/dotnet/fundamentals/networking/http/httpclient)

Pro pokročilejší studium doporučuji následující knihu, která dobře popisuje princip Single Page Apps a Traditional Web Apps a vysvětluje jaké základní technologie jsou k dispozici v .NETu. Kniha sice není určená pro začátečníky, ale nevadí pokud nebudete rozumnět všemu. V knize se například aplikuje Clean Architecture a využívá už některá hotová řešení prostřednitvím nuget balíčků, ale dává náhled na architkturu firemních webových aplikací. V příkladech se také používá například návrhový vzor Repository a Specification. Jeho použití ale v projektech není povinné a nemusí se používat, už samotný DbContext představuje Repository a můžete jej používat přímo.

[ASP.NET Core Application Architecture](https://dotnet.microsoft.com/en-us/learn/aspnet/architecture)

## Ukázka vytvoření webové služby pomocí WebApi

V následujícím příkladu si vytvoříme nejjednodušší webovou službu a zavoláme ji pomocí HttpClienta. Polyglot notebook bohužel **zatím plně nepodporuje** ASP.NET a pro složitější příkladu už bychom musel vytvořit běžný projekt.

Nejprve si nainstalujeme potřebný nuget balíček.

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.AspNetCore,*-*"

Installed Packages Microsoft.DotNet.Interactive.AspNetCore, 1.0.0-beta.23205.1

Loading extensions from `C:\Users\erik\.nuget\packages\microsoft.dotnet.interactive.aspnetcore\1.0.0-beta.23205.1\interactive-extensions\dotnet\Microsoft.DotNet.Interactive.AspNetCore.dll`

Následující kód spustí webovou službu a zobrazí na konzoli na jaké webové adrese a portu je spuštěna.

In [ ]:
#!aspnet

var builder = WebApplication.CreateBuilder();
var app = builder.Build();

int x = 0;
app.MapGet("/", () => $"Hello World {++x}");

await app.StartAsync();

Následující příklad provede HTTP request. V reálné aplikaci bysme nevytvářeli pro každý dotaz nového klienta, více se můžete dočíst v [Guidelines for using HttpClient](https://learn.microsoft.com/en-us/dotnet/fundamentals/networking/http/httpclient-guidelines).

In [ ]:
using(HttpClient client = new HttpClient())
{
    string retezec = await client.GetStringAsync("http://127.0.0.1:5000/");
    Console.WriteLine(retezec);
}

Webovou službu můžeme ukončit následujícím příkazem:

In [ ]:
await app.StopAsync();

## Ukázka Dependency Injection v Minimal WebApi

Všimněte si, že tentokrát nezískáváme referenci v parametru konstruktoru, ale jako parametr metody.

Nejprve vložíme další nuget balíček.

In [2]:
#r "nuget: Microsoft.Extensions.DependencyInjection"

Installed Packages Microsoft.Extensions.DependencyInjection, 7.0.0

In [3]:
public interface INumberGenerator
{
    Task<int> GetNumber();
}

public class RandomNumberGenerator : INumberGenerator
{
    public Task<int> GetNumber()
    {
        return Task.FromResult(Random.Shared.Next(0, 100));
    }
}

In [8]:
#!aspnet

using Microsoft.Extensions.DependencyInjection;

var builder = WebApplication.CreateBuilder();
builder.Services.AddSingleton<INumberGenerator, RandomNumberGenerator>();

var app = builder.Build();

app.MapGet("/", async (INumberGenerator numberGenerator) => 
{
    int number = await numberGenerator.GetNumber();

    return $"Hello World: {number}";
});

await app.StartAsync();

info: Microsoft.Hosting.Lifetime[14]
      Now listening on: http://localhost:5000
info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Production
info: Microsoft.Hosting.Lifetime[0]
      Content root path: c:\Users\erik\source\repos\FAI\OP\Notebooks


Následující příklad provede HTTP request. V reálné aplikaci bysme nevytvářeli pro každý dotaz nového klienta, více se můžete dočíst v [Guidelines for using HttpClient](https://learn.microsoft.com/en-us/dotnet/fundamentals/networking/http/httpclient-guidelines).

In [11]:
using(HttpClient client = new HttpClient())
{
    string retezec = await client.GetStringAsync("http://127.0.0.1:5000/");
    Console.WriteLine(retezec);
}

Hello World: 25


Webovou službu můžeme ukončit následujícím příkazem:

In [12]:
await app.StopAsync();